In [187]:
import json

In [188]:
with open('export_fiches_RNCP_2020-08-03.xml') as fd:
    doc = xmltodict.parse(fd.read())

In [189]:
rncpdict = json.loads(json.dumps(doc))

In [190]:
len(rncpdict['FICHES']['FICHE'])

17763

In [191]:
maliste = []

In [192]:
def recursivedict(level,parent,el,elstring):
    if(type(parent[el]) is dict):
        for el2 in parent[el]:
            elstring=str(el)+'.'+str(el2)
            #print(parent[el][el2])
            #print(el2)
            recursivedict(level+1,parent[el],el2,elstring)
    elif(type(parent[el]) is list):
        if(el not in maliste):
            maliste.append(el)
    else:
        if(el == 0):
            for el2 in parent[el]:
                recursivedict(el,el2)
        else:
            mystr=""
            for i in range(level):
                mystr = mystr+"----"
            #print(mystr+elstring)


In [193]:
for i in range(len(rncpdict['FICHES']['FICHE'])):
    recursivedict(0,rncpdict['FICHES']['FICHE'],i,'')

In [194]:
maliste

['NSF',
 'ROME',
 'BLOC_COMPETENCES',
 'CERTIFICATEUR',
 'PARTENAIRE',
 'FORMACODE',
 'STATISTIQUES_PROMOTION',
 'PUBLICATION_JO',
 'NOUVELLE_CERTIFICATION',
 'ANCIENNE_CERTIFICATION']

In [459]:
def recursivedict2(level,parent,el,elstring,ficheid):
    if(el not in maliste):
        if(type(parent[el]) is dict):
            for el2 in parent[el]:
                if(type(el) is not int):
                    elstring=str(el)+'.'+str(el2)
                else:
                    elstring=str(el2)
                recursivedict2(level+1,parent[el],el2,elstring,ficheid)
        else:
            if(el == 0):
                for el2 in parent[el]:
                    recursivedict2(level+1,parent[el],el2,elstring,ficheid)
            else:
                mydict[elstring] = parent[el]
    else:
        if(type(parent[el]) is dict):
            elmdict = {}
            for elm in parent[el]:
                elmdict[elm] = parent[el][elm]
            if(elmdict != None):
                elmdict['ID_FICHE'] = ficheid
                annexedict[el].append(elmdict)
        else:
            if(type(parent[el]) is list):
                try:
                    for elm in parent[el]:
                        if(type(elm) is dict):
                            elmdict = {}
                            for elm2 in elm:
                                elmdict[elm2] = elm[elm2]
                            if(elmdict != None):
                                elmdict['ID_FICHE'] = ficheid
                                annexedict[el].append(elmdict)
                        else:
                            annexedict[el].append({el: parent[el],'ID_FICHE':ficheid})
                except:
                    print(parent[el])
            else:
                annexedict[el].append({el: parent[el],'ID_FICHE':ficheid})




In [460]:
annexedict = {}
for el in maliste:
    annexedict[el] = []
  

In [461]:
annexedict

{'NSF': [],
 'ROME': [],
 'BLOC_COMPETENCES': [],
 'CERTIFICATEUR': [],
 'PARTENAIRE': [],
 'FORMACODE': [],
 'STATISTIQUES_PROMOTION': [],
 'PUBLICATION_JO': [],
 'NOUVELLE_CERTIFICATION': [],
 'ANCIENNE_CERTIFICATION': []}

In [462]:
arr = []
for i in range(len(rncpdict['FICHES']['FICHE'])):
    mydict = {}
    recursivedict2(0,rncpdict['FICHES']['FICHE'],i,'',rncpdict['FICHES']['FICHE'][i]['ID_FICHE'])
    arr.append(mydict)

In [463]:
df = pd.DataFrame(arr)

In [464]:
df.shape

(17763, 42)

In [465]:
df.to_csv('output/rncp_main.csv',index=False)

In [466]:
for el in annexedict:
    df = pd.DataFrame(annexedict[el])
    df.to_csv('output/rncp_annexe_'+el+'.csv',index=False)